<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/HYPERPARAMETERS_TURNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Time formatter
def time_fmt(sec):
  hours = int(sec/(60*60))
  minutes = int(sec%(60*60)/60)
  seconds = int(sec%60)
  return f"{hours}: {minutes:>02}: {seconds:>05.2f}"

In [ ]:
#Setup Google Colaboratory environment.
from google.colab import drive
try:
  drive.mount('/content/drive', force_remount = True)
  import tensorflow as tf
  COLAB = True
  print(f"You are using Google Colab with Tensorflow version---->{tf.__version__}")
except:
  COLAB = False
  print("You are not Connected!!")

Mounted at /content/drive
You are using Google Colab with Tensorflow version---->2.3.0


In [ ]:
#Loading the necessary packs
import time
import os
import statistics
import pandas as pd
import numpy as np
from scipy.stats import zscore
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, PReLU,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit



In [ ]:
#Loading the data from the url
data = pd.read_csv("https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv", na_values = ['NAN','?'])

In [ ]:
display(data.head())

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a


In [ ]:
data.drop('id', axis = 1, inplace = True)

In [ ]:
data['income'] = data['income'].fillna(data['income'].median())

In [ ]:
cols = data.columns.drop(['area','job','subscriptions','product'])

In [ ]:
for col in cols:
  data[col] = zscore(data[col])

In [ ]:
jobd = pd.get_dummies(data['job'], prefix = 'job')
aread = pd.get_dummies(data['area'], prefix = 'area')
y = data['product']
y = pd.get_dummies(y, prefix = 'product')
data.drop(columns = ['job', 'product', 'area'], axis = 1, inplace=True)

In [ ]:
data = pd.concat([data, aread,jobd], axis = 1)

In [ ]:
display(data.head())

,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,area_a,area_b,area_c,area_d,job_11,job_al,job_am,job_ax,job_bf,job_by,job_cv,job_de,job_dz,job_e2,job_f8,job_gj,job_gv,job_kd,job_ke,job_kl,job_kp,job_ks,job_kw,job_mm,job_nb,job_nn,job_ob,job_pe,job_po,job_pq,job_pz,job_qp,job_qw,job_rn,job_sa,job_vv,job_zz
0,-0.607550,-0.664918,1,-0.048411,-0.215764,-0.314089,0.854321,0.079279,-0.465765,-1.120315,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0.338053,-0.207748,2,-0.266765,0.196869,-0.915161,1.394432,-0.075010,-1.445372,0.682945,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.184205,1.127906,1,-0.988286,-0.714362,-0.078604,-0.495957,0.850727,1.055205,-0.373087,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,-0.526467,-0.440815,1,-0.684488,-0.542432,-1.216347,1.124377,0.130709,-0.775115,0.466401,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-2.851675,1.638861,3,-0.955058,-0.473660,-1.017291,-2.116291,-1.772196,0.642739,-1.137090,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x = data.values

In [ ]:
y = y.values

In [ ]:
x[0:10]


array([[-0.60754957, -0.66491815,  1.        , -0.04841068, -0.21576413,
        -0.31408854,  0.85432106,  0.07927915, -0.46576475, -1.12031509,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [ 0.33805295, -0.20774798,  2.        , -0.26676549,  0.19686897,
        -0.91516076,  1.39443237, -0.07501047, -1.44537236,  0.68294487,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.     

In [ ]:
y[0:10]

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0]], dtype=uint8)

In [ ]:
#NNT model with hyperparameters turning

In [ ]:
def hyper_turn_nnt(dropout,lr, neuronPct,neuronShrink):
  SPLITS = 5
  boots_samples = StratifiedShuffleSplit(n_splits = SPLITS, test_size = 0.1)
  count = 0
  av_epochs = []
  av_scores = []
  neurons_count = int(neuronPct * 5000)
  #Looping through the bootstrap samples
  for train, test in boots_samples.split(x, y):
    time_start = time.time()
    count+=1
    x_train = x[train]
    x_test = x[test]
    y_train = y[train]
    y_test = y[test]

    #Model architecture
    model = Sequential()
    layers = 0
    while neurons_count > 50 and layers < 10:
      if layers == 0:
        model.add(Dense(neurons_count, input_dim = x.shape[1], activation = PReLU()))
      else:
        model.add(Dense(neurons_count, activation = PReLU()))
      model.add(Dropout(dropout))
      neurons_count = neurons_count * neuronShrink
      model.add(Dense(y.shape[1], activation = 'softmax'))
      model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr = lr))
      info = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, verbose = 1, mode = 'auto',
                           patience = 100, restore_best_weights = True)
      model.fit(x_train, y_train, validation_data = (x_test, y_test), verbose = 2, callbacks = [info], epochs = 1000)
      epoc = info.stopped_epoch
      av_epochs.append(epoc)
      pred = model.predict(x_test)
      y_match = np.argmax(y_test, axis = 1)
      score1 = metrics.log_loss(y_match, pred)
      av_scores.append(score1)
      mean1 = statistics.mean(av_scores)
      mean2 = statistics.mean(av_epochs)
      mean_dev = statistics.pstdev(av_scores)
    tensorflow.keras.backend.clear_session()
    return f"Mean_score_loss:{mean1}\n Average_epoch: {mean2}\n Mean_dev_score_loss: {mean_dev}"
print(hyper_turn_nnt(dropout = 0.2,lr = 1e-03, neuronPct = 0.1,neuronShrink = 0.2))


    
      

Epoch 1/1000
57/57 - 0s - loss: 1.1855 - val_loss: 0.8942
Epoch 2/1000
57/57 - 0s - loss: 0.8292 - val_loss: 0.7897
Epoch 3/1000
57/57 - 0s - loss: 0.7378 - val_loss: 0.7498
Epoch 4/1000
57/57 - 0s - loss: 0.6984 - val_loss: 0.7242
Epoch 5/1000
57/57 - 0s - loss: 0.6624 - val_loss: 0.7253
Epoch 6/1000
57/57 - 0s - loss: 0.6469 - val_loss: 0.7151
Epoch 7/1000
57/57 - 0s - loss: 0.6173 - val_loss: 0.6985
Epoch 8/1000
57/57 - 0s - loss: 0.6139 - val_loss: 0.7119
Epoch 9/1000
57/57 - 0s - loss: 0.5916 - val_loss: 0.6983
Epoch 10/1000
57/57 - 0s - loss: 0.5842 - val_loss: 0.7067
Epoch 11/1000
57/57 - 0s - loss: 0.5770 - val_loss: 0.6943
Epoch 12/1000
57/57 - 0s - loss: 0.5549 - val_loss: 0.6947
Epoch 13/1000
57/57 - 0s - loss: 0.5502 - val_loss: 0.7054
Epoch 14/1000
57/57 - 0s - loss: 0.5382 - val_loss: 0.6999
Epoch 15/1000
57/57 - 0s - loss: 0.5267 - val_loss: 0.6865
Epoch 16/1000
57/57 - 0s - loss: 0.5199 - val_loss: 0.6903
Epoch 17/1000
57/57 - 0s - loss: 0.5199 - val_loss: 0.7012
Epoch 